# 🗺️ Booking Reviews


## 🧰 Librerías e importaciones

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## 🌐 Configuración del navegador y apertura de la URL

In [10]:
chrome_options = Options()
#chrome_options.add_argument("--headless")  # Quita esto si quieres ver el navegador
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

In [12]:
driver = webdriver.Chrome(service=Service(), options=chrome_options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
actions = ActionChains(driver)
wait = WebDriverWait(driver, 10)

In [13]:
df = pd.read_csv("booking_canarias_limpio_agosto.csv")

In [16]:
comentarios_df = pd.DataFrame(columns=["titulo", "comentario", "review_score", "region", "categoria", "fecha"])

## ⬇️ Preparación: scroll y expansión

In [19]:
def scroll_lentamente(pixeles=300, pausas=0.5, repeticiones=5):
    for _ in range(repeticiones):
        driver.execute_script(f"window.scrollBy(0, {pixeles});")
        time.sleep(pausas)

## 🔎 Scraping 

In [22]:
def scrapear_experiencia(url):
    comentarios = []

    try:
        driver.get(url)
        time.sleep(3)  # Aumentar tiempo de espera inicial

        # Cerrar cookies si aparece
        try:
            aceptar_cookies = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
            aceptar_cookies.click()
            time.sleep(1)
        except:
            pass

        # Extraer título - CORREGIR SELECTOR
        try:
            # Probar varios selectores posibles para el título
            selectors_titulo = [
                'h1[data-testid="title"]',
                'h1.pp-header__title',
                '#hp_hotel_name',
                'h1'
            ]
            
            titulo = "Sin título"
            for selector in selectors_titulo:
                try:
                    titulo_element = driver.find_element(By.CSS_SELECTOR, selector)
                    titulo = titulo_element.text.strip()
                    break
                except:
                    continue
        except:
            titulo = "Sin título"

        print(f"Título encontrado: {titulo}")

        # Scroll para cargar contenido
        scroll_lentamente(pixeles=500, pausas=0.3, repeticiones=3)

        # Buscar sección de comentarios - CORREGIR SELECTORES
        try:
            # Múltiples selectores para encontrar la sección de reviews
            review_selectors = [
                '[data-testid="PropertyReviewsRegionBlock"]',
                '[data-testid="review-section"]',
                '[id*="reviews"]',
                '.reviews-section',
                '[data-testid="guest-reviews"]'
            ]
            
            ver_pagina = None
            for selector in review_selectors:
                try:
                    ver_pagina = driver.find_element(By.CSS_SELECTOR, selector)
                    print(f"Sección de reviews encontrada con selector: {selector}")
                    break
                except:
                    continue
                    
            if ver_pagina is None:
                print("No se encontró la sección de reviews")
                return []

            # Scroll hacia la sección de reviews
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", ver_pagina)
            time.sleep(2)

            # Buscar botón de "ver todos los comentarios" - MÚLTIPLES SELECTORES
            button_selectors = [
                '[data-testid="fr-read-all-reviews"]',
                'button[data-testid="read-all-reviews"]',
                'a[data-testid="read-all-reviews"]',
                'button:contains("Ver todos")',
                'a:contains("Ver todos")',
                'button:contains("comentarios")',
                'a:contains("comentarios")'
            ]
            
            ver_comentarios_btn = None
            for selector in button_selectors:
                try:
                    if ':contains(' in selector:
                        # Para selectores con contains, usar XPath
                        xpath = f"//button[contains(text(), 'Ver todos')] | //a[contains(text(), 'Ver todos')] | //button[contains(text(), 'comentarios')] | //a[contains(text(), 'comentarios')]"
                        ver_comentarios_btn = driver.find_element(By.XPATH, xpath)
                    else:
                        ver_comentarios_btn = ver_pagina.find_element(By.CSS_SELECTOR, selector)
                    print(f"Botón encontrado con selector: {selector}")
                    break
                except:
                    continue
            
            if ver_comentarios_btn is None:
                print("No se encontró el botón de 'ver todos los comentarios'")
                # Intentar buscar en toda la página
                try:
                    xpath_general = "//button[contains(text(), 'Ver')] | //a[contains(text(), 'Ver')] | //button[contains(text(), 'comentarios')] | //a[contains(text(), 'comentarios')] | //button[contains(text(), 'reseñas')] | //a[contains(text(), 'reseñas')]"
                    ver_comentarios_btn = driver.find_element(By.XPATH, xpath_general)
                    print("Botón encontrado con búsqueda general")
                except:
                    print("No se pudo encontrar ningún botón de comentarios")
                    return []

            # Hacer click en el botón
            try:
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", ver_comentarios_btn)
                time.sleep(1)
                
                # Intentar click normal primero
                try:
                    ver_comentarios_btn.click()
                except:
                    # Si falla, usar JavaScript click
                    driver.execute_script("arguments[0].click();", ver_comentarios_btn)
                
                time.sleep(3)
                print("Click en botón de comentarios exitoso")
                
            except Exception as e:
                print(f"Error haciendo click en botón: {e}")
                return []

        except Exception as e:
            print(f"Error buscando sección de comentarios: {e}")
            return []

        # Esperar que cargue el modal o nueva página
        try:
            # Múltiples selectores para el contenedor de reviews
            modal_selectors = [
                '[data-testid="review-list-container"]',
                '[data-testid="reviews-modal"]',
                '.review-list',
                '[id*="review"]'
            ]
            
            review_modal = None
            for selector in modal_selectors:
                try:
                    review_modal = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
                    print(f"Modal/contenedor de reviews encontrado: {selector}")
                    break
                except:
                    continue
                    
            if review_modal is None:
                print("No se encontró el modal/contenedor de reseñas")
                return []
                
        except Exception as e:
            print(f"Error esperando modal de reseñas: {e}")
            return []

        # Loop para recorrer páginas de comentarios
        page_count = 0
        max_pages = 50  # Límite de seguridad
        
        while page_count < max_pages:
            print(f"Procesando página {page_count + 1}")
            
            # Selectores para tarjetas de review
            card_selectors = [
                '[data-testid="review-card"]',
                '.review-card',
                '[data-testid="review"]',
                '.review-item'
            ]
            
            review_items = []
            for selector in card_selectors:
                try:
                    review_items = review_modal.find_elements(By.CSS_SELECTOR, selector)
                    if review_items:
                        print(f"Encontradas {len(review_items)} reviews con selector: {selector}")
                        break
                except:
                    continue
            
            if not review_items:
                print("No se encontraron elementos de review")
                break
                
            for item in review_items:
                try:
                    # Extraer puntuación
                    score = None
                    score_selectors = [
                        '[data-testid="review-score"]',
                        '.review-score',
                        '[aria-label*="puntuación"]',
                        '[aria-label*="score"]'
                    ]
                    
                    for score_sel in score_selectors:
                        try:
                            score_element = item.find_element(By.CSS_SELECTOR, score_sel)
                            score_text = score_element.get_attribute("aria-label") or score_element.text
                            # Extraer número de la puntuación
                            import re
                            score_match = re.search(r'(\d+(?:\.\d+)?)', score_text)
                            if score_match:
                                score = float(score_match.group(1))
                                break
                        except:
                            continue

                    # Extraer texto del comentario
                    texto = ""
                    text_selectors = [
                        '[data-testid="review-positive-text"]',
                        '[data-testid="review-negative-text"]',
                        '.review-text',
                        '[data-testid="review-content"]'
                    ]
                    
                    for text_sel in text_selectors:
                        try:
                            text_elements = item.find_elements(By.CSS_SELECTOR, text_sel)
                            for element in text_elements:
                                texto += element.text.strip() + " "
                        except:
                            continue
                    
                    texto = texto.strip()

                    # Extraer metadatos (país, categoría, fecha)
                    pais = ""
                    categoria = ""
                    fecha = ""
                    
                    try:
                        # Buscar elementos de fecha/metadata
                        metadata_selectors = [
                            '[data-testid="review-date"]',
                            '.review-date',
                            '.review-metadata'
                        ]
                        
                        for meta_sel in metadata_selectors:
                            try:
                                metadata_elements = item.find_elements(By.CSS_SELECTOR, meta_sel)
                                for elem in metadata_elements:
                                    text = elem.text.strip()
                                    if text:
                                        # Intentar identificar qué tipo de dato es
                                        if any(month in text.lower() for month in ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 
                                                                                  'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre',
                                                                                  'january', 'february', 'march', 'april', 'may', 'june',
                                                                                  'july', 'august', 'september', 'october', 'november', 'december']):
                                            fecha = text
                                        elif len(text) < 50 and not fecha:  # Posible país o categoría
                                            if not pais:
                                                pais = text
                                            elif not categoria:
                                                categoria = text
                                break
                            except:
                                continue
                                
                    except Exception as e:
                        print(f"Error extrayendo metadatos: {e}")

                    # Solo agregar si tiene contenido útil
                    if texto or score is not None:
                        comentarios.append({
                            "titulo": titulo,
                            "review_score": score,
                            "comentario": texto,
                            "pais": pais,
                            "categoria": categoria,
                            "fecha": fecha
                        })

                except Exception as e:
                    print(f"Error procesando review individual: {e}")
                    continue

            # Buscar botón "Siguiente" con múltiples estrategias
            try:
                next_button_found = False
                
                # Selectores para botón siguiente
                next_selectors = [
                    'button[aria-label="Página siguiente"]',
                    'button[aria-label="Next page"]',
                    'button:contains("Siguiente")',
                    'a:contains("Siguiente")',
                    '[data-testid="pagination-next"]',
                    '.pagination-next',
                    'button[aria-label*="siguiente"]'
                ]
                
                for next_sel in next_selectors:
                    try:
                        if ':contains(' in next_sel:
                            xpath = "//button[contains(text(), 'Siguiente')] | //a[contains(text(), 'Siguiente')] | //button[contains(@aria-label, 'siguiente')] | //button[contains(@aria-label, 'Next')]"
                            next_btn = driver.find_element(By.XPATH, xpath)
                        else:
                            next_btn = driver.find_element(By.CSS_SELECTOR, next_sel)
                        
                        # Verificar si está habilitado
                        if next_btn.get_attribute("disabled") is not None or "disabled" in next_btn.get_attribute("class") or "":
                            print("Botón siguiente deshabilitado - fin del paginado")
                            return comentarios
                        
                        # Scroll hacia el botón
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", next_btn)
                        time.sleep(1)
                        
                        # Intentar click
                        try:
                            wait.until(EC.element_to_be_clickable(next_btn))
                            next_btn.click()
                            print("Click en 'Siguiente' exitoso")
                            next_button_found = True
                            break
                        except:
                            # Intentar con JavaScript
                            driver.execute_script("arguments[0].click();", next_btn)
                            print("Click con JavaScript en 'Siguiente' exitoso")
                            next_button_found = True
                            break
                            
                    except:
                        continue
                
                if not next_button_found:
                    print("No se encontró botón 'Siguiente' - fin del paginado")
                    break
                    
                time.sleep(2)  # Esperar que cargue la siguiente página
                page_count += 1
                
            except Exception as e:
                print(f"Error en paginación: {e}")
                break

    except Exception as e:
        print(f"Error general en {url}: {e}")

    return comentarios

## 💾 Guardado a CSV

In [ ]:
for idx, row in df.iterrows():
    print(f"\n=== Procesando {idx + 1}/{len(df)}: {row['url']} ===")
    try:
        resultados = scrapear_experiencia(row["url"])
        if resultados:
            print(f"Obtenidos {len(resultados)} comentarios")
            comentarios_df = pd.concat([comentarios_df, pd.DataFrame(resultados)], ignore_index=True)
        else:
            print("No se obtuvieron comentarios")
    except Exception as e:
        print(f"Error procesando hotel {idx + 1}: {e}")
    
    time.sleep(2)

# Guardar resultados
comentarios_df.to_csv("hoteles_canarias_completos.csv", index=False)
print(f"\nArchivo final guardado con {len(comentarios_df)} comentarios totales.")

# Cerrar navegador
driver.quit()


=== Procesando 1/490: https://www.booking.com/hotel/es/dona-rose-surf-hostel.es.html?label=canary-islands-_cErSRutz9ZwI2zHpFaB0gS349826210110%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-319830933545%3Alp1005493%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YTiRJUvwM0AZDoPI6XBbFtM&aid=1610834&ucfs=1&arphpl=1&checkin=2025-08-19&checkout=2025-08-20&dest_id=754&dest_type=region&group_adults=1&req_adults=1&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=7ed95cd4918a0151&srepoch=1755436429&all_sr_blocks=1265626006_401380197_1_0_0&highlighted_blocks=1265626006_401380197_1_0_0&matching_block_id=1265626006_401380197_1_0_0&sr_pri_blocks=1265626006_401380197_1_0_0__1570&from=searchresults ===
Título encontrado: Playa
Doña Rose Surf Hostel
Sección de reviews encontrada con selector: [data-testid="PropertyReviewsRegionBlock"]
Botón encontrado con selector: [data-testid="fr-read-all-reviews"]
Click en botón de comentarios exitoso
Modal/contenedor de reviews